In [1]:
import pandas as pd

In [2]:
df_train = pd.read_csv('../data/custom/df_train.csv')

In [7]:
df_train.head(5)

,DATETIME,ND_TARGET,ND_PREV,TSD,ENGLAND_WALES_DEMAND,EMBEDDED_WIND_GENERATION,EMBEDDED_WIND_CAPACITY,EMBEDDED_SOLAR_GENERATION,EMBEDDED_SOLAR_CAPACITY,NON_BM_STOR,...,temp_liverpool,temp_min_london,temp_min_bath,temp_min_liverpool,temp_max_london,temp_max_bath,temp_max_liverpool,humidity_london,humidity_bath,humidity_liverpool
0,2019-01-02 00:00:00,24147,23808,25291,22393,2548,5918,0,13052,0,...,274.72,277.17,276.83,271.85,279.12,278.89,276.30,71,87,79
1,2019-01-02 01:00:00,22316,24147,25495,22689,2396,5918,0,13052,0,...,274.15,276.06,275.57,271.29,279.12,278.89,275.75,74,85,79
2,2019-01-02 02:00:00,20958,22316,24346,20979,2236,5918,0,13052,0,...,273.73,277.17,274.98,271.29,279.32,278.33,275.01,72,84,80
3,2019-01-02 03:00:00,20331,20958,23695,19433,2081,5918,0,13052,0,...,273.08,277.17,273.87,270.74,279.21,277.77,274.59,69,85,82
4,2019-01-02 04:00:00,20138,20331,23080,18687,1958,5918,0,13052,0,...,272.77,277.17,273.78,270.74,279.12,277.22,274.03,68,86,82
